# Python for Machine Learning

### *Session \#4*


### Helpful shortcuts
---

**SHIFT** + **ENTER** ----> Execute Cell

**UP/DOWN ARROWS** --> Move cursor between cells (then ENTER to start typing)

**TAB** ----> See autocomplete options

**ESC** then **b** ----> Create Cell 

**ESC** then **dd** ----> Delete Cell

**\[python expression\]?** ---> Explanation of that Python expression

**ESC** then **m** then __ENTER__ ----> Switch to Markdown mode

## I. Logistic Regression

### Warm Ups

*Type the given code into the cell below*

---

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from yellowbrick.classifier import ConfusionMatrix, ClassPredictionError, ROCAUC, PrecisionRecallCurve
from yellowbrick.target import class_balance

from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler

df = pd.read_csv('heart_attack.csv')

**Create and fit classifier**: 

In [ ]:
def assign_tts(df, model, tgt_txt, features='all', state=None):
    if features == 'all':
        X=df.drop(columns=[tgt_txt])
    elif isinstance(features, list):
        X=df[features]
    else:
        raise ValueError('Features should either be \'all\' or a list')
    y=df[tgt_txt]
    tts = train_test_split(X,y, random_state=state)
    model.X_train = tts[0]
    model.X_test = tts[1]
    model.y_train = tts[2]
    model.y_test = tts[3]
    return None

In [ ]:
model_Iw = LogisticRegression()
assign_tts(df, model_Iw, 'heart_attack', features=['age'], state=1)
model_Iw.fit(model_Iw.X_train, model_Iw.y_train)

**Use model to classify**:

In [ ]:
model_Iw.predict(model_Iw.X_test)

**Use model to get probabilities**:

In [ ]:
model_Iw.predict_proba(model_Iw.X_test)

### Exercises
---

**1. Graph these probabilities with** `plt.plot()` 

In [ ]:
probabilities = np.array([0.001, 0.002, 0.004, 0.009, 
                         0.018, 0.036, 0.072, 0.139, 
                         0.251, 0.41 , 0.59 , 0.749, 
                         0.861, 0.928, 0.964, 0.982, 
                         0.991, 0.996, 0.998, 0.999])

**2. Now convert these probabilities into log-odds.**

**First divide** `probabilities` **by** `1 - probabilities` 

**Use** `np.log()` **afterwards, and** `.plot()` **the result.**

**3. Let's go from log-odds back to probabilities now!**

**Copy/paste the slope from** `model.coef_` **and the intercept from** `model.intercept_`

**Save them as** `slope` **and** `intercept`

**4. Plot the underlying linear model using plt.plot().**

**First feed in** `X_test` **as the x-axis and** `X_test*slope + intercept` **as the y-axis**

**5. Use** `plt.scatter` **to plot** `X_test` **and** `y_test`, **and also to plot** `curve_x` and `curve_y` **which show the curve of the logistic classifier**

In [ ]:
curve_x = np.linspace(-10, 150, 40).reshape(-1, 1)
curve_y = [a for a,b in model_Iw.predict_proba(curve_x)]

## II. ROC Curves and Class Imbalance

### Warm Ups

*Type the given code into the cell below*

---

**Create and fit classifier**: 

In [ ]:
model_roc = ROCAUC(LogisticRegression(max_iter=int(1e4)))
assign_tts(df, model_roc, 'heart_attack', state=1)
model_roc.fit(model_roc.X_train, model_roc.y_train)
model_roc.score(model_roc.X_test, model_roc.y_test)
model_roc.show()

**Create ClassBalance visualization:**

In [ ]:
class_balance(df['heart_attack'])

**Plot the Precision Recall Curve**: 

In [ ]:
model_prc = PrecisionRecallCurve(model_Iw)
assign_tts(df, model_prc, 'heart_attack', features=['age'], state=1)
model_prc.fit(model_prc.X_train, model_prc.y_train)
model_prc.score(model_prc.X_test, model_prc.y_test)
model_prc.show()
plt.show()

### Exercises
---

**1. Interpret the ROC curve below. What is the highest sensitivity we can reach while keeping false positives under 20% (ie. specificity > 0.8)?**

**If we care about both classes equally, what sensitivity and specificity should we choose?**

![image.png](../images/roc.png)

**2. If you were creating a machine learning model to catch credit card fraud, would you use an ROC curve or a precision-recall curve?**

**3. Train and plot an ROC curve with a** `KNeighborsClassifier` **model and a** `LogisticRegression` **model. Which model performs better with this data, according to the AUC?** 

Hint: To help deal with limited data, you may need to pass `max_iter=10000` when creating your LogisticRegression model

**4. Do the same thing but replace the** `ROCAUC` **visualizers with** `PrecisionRecallCurve` **visualizers and rerun to get a minority-class focused view on performance.**

**5. Let's examine more severe class imbalance.**

**Run the code below to drop most of the positive cases, then split the data into X and y again**

**Create a** `class_balance` **visualization to verify that the classes are now very imbalanced.**

In [ ]:
df_drop = df.drop(df.query('heart_attack == 1').sample(n=400).index)

# Add your code down here

**6. Rewrite and run code similar to that which you wrote for Question 3, but this time use `df_drop` as your data source. Which model's performance has suffered more? Why?**

Hint: To help deal with limited data, you may need to pass `max_iter=10000` when creating your LogisticRegression model

**7. Do the same thing but replace the** `ROCAUC` **visualizers with** `PrecisionRecallCurve` **visualizers and rerun to get a minority-class focused view on performance.**

## III. Stratified Sampling and Oversampling

### Warm Ups

*Type the given code into the cell below*

---

**Use stratified sampling:**
```python
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
```

**Use RandomOverSampler to balance data:**
```python
sampler = RandomOverSampler()
sampler.fit_resample(X_train, y_train)
```

**Use RandomOverSampler in pipeline:**
```python
model = make_pipeline(RandomOverSampler(), LogisticRegression())
```

### Exercises
---

**1. Inside the for-loop, call** `train_test_split()` **WITHOUT the stratify parameter**

**Then within the for-loop call** `y_test.sum()` **to count the number of positive cases.**

**Rerun with the stratify parameter set to** `y`

In [ ]:
for i in range(10):
    
    # Add your code here

**2. Create a** `RandomOverSampler()` **and use .fit_resample() on X_train and y_train**

**This will return two arrays -- the rebalanced versions of** `X_train` **and** `y_train` 

**Take the mean of the new rebalanced** `y_train` **to show that it's balanced**

**3. Fit a** `LogisticRegression()` **model to the training data, and use it to plot a ConfusionMatrix**

**What is the accuracy and sensitivity of the model?**

**4. Create a pipeline with a** `RandomOverSampler` **and** `LogisticRegression()` **and fit it to the training data**

**What is the accuracy and sensitivity of the new model?**